# Extraction of models from DeciContas.br Dataset

In [2]:
import os
import json
import pymssql

import pandas as pd
import numpy as np


from pprint import pprint
from langchain_openai import  AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv
from sqlalchemy import (
    create_engine, Column, Integer, String, Float, Date, Boolean, Text, JSON
)
from sqlalchemy.orm import declarative_base, sessionmaker

from tools.prompt import generate_few_shot_ner_prompts, extract_obrigacao, extract_recomendacao, insert_obrigacao, insert_recomendacao
from tools.schema import (
    NERDecisao,
    Obrigacao,
    Recomendacao
)

from tools.models import (
    Base, ObrigacaoORM, RecomendacaoORM
)

from datetime import timedelta


load_dotenv()


gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)

'''
gpt4turbo = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.0
)
'''

extractor_gpt4turbo = gpt4turbo.with_structured_output(NERDecisao, include_raw=False, method="json_schema")
extractor_obrigacao_gpt4turbo = gpt4turbo.with_structured_output(
    Obrigacao, include_raw=False, method="json_schema")
extractor_recomendacao_gpt4turbo = gpt4turbo.with_structured_output(
    Recomendacao, include_raw=False, method="json_schema")



def get_connection(db: str = 'processo'):
    server = os.getenv("SQL_SERVER_HOST")
    user = os.getenv("SQL_SERVER_USER")
    password = os.getenv("SQL_SERVER_PASS")
    port = os.getenv("SQL_SERVER_PORT", "1433")  # default MSSQL port
    database = db

    return pymssql.connect(server, user, password, database, port)


c:\Users\05911205424\Documents\Dev\cgad\venv\Lib\site-packages\langchain_openai\chat_models\base.py:1927: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [43]:
import importlib
importlib.reload(__import__('tools.schema'))

<module 'tools' from 'c:\\Users\\05911205424\\Documents\\Dev\\cgad\\etl\\tools\\__init__.py'>

# From DB Processo

In [2]:
meses = (10,11)
sql_decisoes = f'''
SELECT *
FROM processo.dbo.vw_ia_votos_acordaos_decisoes d
WHERE YEAR(DataSessao) = 2025
AND MONTH(DataSessao) IN {meses}
AND NOT EXISTS(
	SELECT *
	FROM BdDIP.dbo.Obrigacao o 
	WHERE d.IdComposicaoPauta = o.IdComposicaoPauta
	AND d.idVotoPauta = o.idVotoPauta 
)
AND NOT EXISTS(
	SELECT *
	FROM BdDIP.dbo.Recomendacao r 
	WHERE d.IdComposicaoPauta = r.IdComposicaoPauta
	AND d.idVotoPauta = r.idVotoPauta 
)


'''
df_decisoes = pd.read_sql(sql_decisoes, get_connection())
df_decisoes['ner_decisao'] = ''

C:\Users\05911205424\AppData\Local\Temp\ipykernel_13108\2427677665.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_decisoes = pd.read_sql(sql_decisoes, get_connection())


In [3]:
len(df_decisoes)

436

In [6]:
resultados = []
errors = []
for index, row in df_decisoes.iterrows():
    if index % 10 == 0 and index > 0:
        print(f"Processando instância {index} de {len(df_decisoes)}")
        df_decisoes.to_pickle("decisoes_ner_4T2025.pkl")

    try:
        prompt_with_few_shot = generate_few_shot_ner_prompts(row['texto_acordao'])
        
        result = extractor_gpt4turbo.invoke(prompt_with_few_shot)
        df_decisoes.at[index, 'ner_decisao'] = result
        resultados.append(result)
    except Exception as e:
        errors.append({
            'index': index,
            'error': str(e),
            'row': row.to_dict()
        })


Processando instância 10 de 436
Processando instância 20 de 436
Processando instância 30 de 436
Processando instância 40 de 436
Processando instância 50 de 436
Processando instância 60 de 436
Processando instância 70 de 436
Processando instância 80 de 436
Processando instância 90 de 436
Processando instância 100 de 436
Processando instância 110 de 436
Processando instância 120 de 436
Processando instância 130 de 436
Processando instância 140 de 436
Processando instância 150 de 436
Processando instância 160 de 436
Processando instância 170 de 436
Processando instância 180 de 436
Processando instância 190 de 436
Processando instância 200 de 436
Processando instância 210 de 436
Processando instância 220 de 436
Processando instância 230 de 436
Processando instância 240 de 436
Processando instância 250 de 436
Processando instância 260 de 436
Processando instância 270 de 436
Processando instância 280 de 436
Processando instância 290 de 436
Processando instância 300 de 436
Processando instânc

In [7]:
import re

def slugify(s):
    s = s.lower()
    s = re.sub(r'[^\w\s-]', '', s)
    s = re.sub(r'[\s]+', '_', s)
    return s

df_decisoes.columns = [slugify(col) for col in df_decisoes.columns]
df_decisoes.to_pickle("decisoes_ner_4T2025.pkl")

## Carregando pickle

In [3]:
df_decisoes = pd.read_pickle("decisoes_ner_4T2025.pkl")

In [4]:
len(df_decisoes)

436

In [5]:
sql_orgaos = f'''
SELECT p.idprocesso,
CONCAT(p.numero_processo, '/', p.ano_processo) as processo,
o.nome as orgao_responsavel, 
o.IdOrgao as id_orgao_responsavel,
gp.Nome as nome_responsavel,
gp.Documento as documento_responsavel,
gp.TipoPessoa  as tipo_responsavel,
gp.IdPessoa as id_pessoa
FROM processo.dbo.Processos p
INNER JOIN processo.dbo.Orgaos o ON p.IdOrgaoEnvolvido = o.IdOrgao 
INNER JOIN processo.dbo.Pro_ProcessosResponsavelDespesa pprd ON pprd.IdProcesso = p.IdProcesso 
INNER JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = pprd.IdPessoa 
WHERE p.idprocesso IN ({', '.join([f"{proc}" for proc in df_decisoes['idprocesso'].unique()])})
'''
df_orgaos = pd.read_sql(sql_orgaos, get_connection())

C:\Users\05911205424\AppData\Local\Temp\ipykernel_10784\4281826205.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orgaos = pd.read_sql(sql_orgaos, get_connection())


In [6]:
df_decisoes = df_decisoes.merge(df_orgaos, how='left', left_on='idprocesso', right_on='idprocesso', suffixes=('', '_y'))

In [ ]:

engine = create_engine(
    f"mssql+pymssql://{os.getenv("SQL_SERVER_USER")}:{os.getenv("SQL_SERVER_PASS")}@{os.getenv("SQL_SERVER_HOST")}/{os.getenv("SQL_SERVER_DB")}",
    echo=True
)
#engine = create_engine("sqlite:///dataset/sqlite/obrigacoes.db", echo=True)
SessionLocal = sessionmaker(bind=engine)



In [8]:
df_decisoes.columns

Index(['idvoto', 'monocratica', 'codigo_tipo_processo', 'descricao',
       'tipovoto', 'votoescolhido', 'idinformacao_voto', 'info_arquivo_voto',
       'idcomposicaopauta', 'codigocamara', 'numero_sessao', 'ano_sessao',
       'datasessao', 'datadecisaomonocratica', 'dataencerramentosessao',
       'numeroresultado', 'anoresultado', 'resultadotipo', 'idapreciacao',
       'idtiporecurso', 'idtipovotacao', 'decisao', 'idvotopauta',
       'idvotodecisao', 'nomerelator', 'setor', 'ementa', 'assunto',
       'numeroprocesso', 'anoprocesso', 'nomerelatorprocesso', 'idprocesso',
       'interessado', 'orgaoorigem', 'divergente_de_idvoto',
       'isvotodivergente', 'idvotoconcordado', 'relatorio',
       'fundamentacaovoto', 'conclusao', 'artigo', 'texto_acordao',
       'setorvoto', 'descricaotipovoto', 'ner_decisao', 'processo',
       'orgao_responsavel', 'id_orgao_responsavel', 'nome_responsavel',
       'documento_responsavel', 'tipo_responsavel', 'id_pessoa'],
      dtype='object')

In [9]:
obrigacoes = [(d.idcomposicaopauta, d.idvoto, d.idvotopauta, d.datasessao, d['ner_decisao'].obrigacoes) 
              for _,d in df_decisoes[['idcomposicaopauta', 'idvotopauta', 'idvoto', 'datasessao', 'ner_decisao']].iterrows() if d['ner_decisao'].obrigacoes != []]
df_ob = pd.DataFrame(obrigacoes, columns=['idcomposicaopauta', 'idvoto', 'idvotopauta', 'datasessao', 'obrigacoes'])
df_ob['idvotopauta'] = df_ob['idvotopauta'].astype('int64')

In [10]:
recomendacoes = [(d.idcomposicaopauta, d.idvoto, d.idvotopauta, d.datasessao, d['ner_decisao'].recomendacoes) 
              for _,d in df_decisoes[['idcomposicaopauta', 'idvotopauta', 'idvoto', 'datasessao', 'ner_decisao']].iterrows() if d['ner_decisao'].recomendacoes != []]
df_rec = pd.DataFrame(recomendacoes, columns=['idcomposicaopauta', 'idvoto', 'idvotopauta', 'datasessao', 'recomendacoes'])
df_rec['idvotopauta'] = df_rec['idvotopauta'].astype('int64')

In [ ]:
group_cols = [
        'idprocesso', 'numeroprocesso', 'anoprocesso', 'idcomposicaopauta', 
        'idvotopauta', 'numero_sessao', 'ano_sessao', 'datasessao', 'relatorio',
        'fundamentacaovoto', 'conclusao', 'texto_acordao', 'orgao_responsavel', 'id_orgao_responsavel',
]
person_cols = ['nome_responsavel', 'documento_responsavel', 'tipo_responsavel', 'id_pessoa']

df_decisoes_aug = df_decisoes.groupby(group_cols, dropna=False).apply(
    lambda x: pd.Series({'responsaveis': x[person_cols].apply(
        lambda y: y.dropna().to_dict(), axis=1
    ).tolist()})
).reset_index()

C:\Users\05911205424\AppData\Local\Temp\ipykernel_10784\1014293047.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_decisoes_aug = df_decisoes.groupby(group_cols, dropna=False).apply(


In [13]:
df_ob = df_ob[['idcomposicaopauta', 'idvotopauta', 'obrigacoes']].merge(df_decisoes_aug, on=['idcomposicaopauta', 'idvotopauta'], how='left')
df_ob['datasessao'] = pd.to_datetime(df_ob['datasessao'], errors='coerce')
df_ob['n_obrigacoes'] = df_ob['obrigacoes'].apply(lambda x: len(x) if isinstance(x, list) else 0)


In [14]:
df_rec = df_rec[['idcomposicaopauta', 'idvotopauta', 'recomendacoes']].merge(df_decisoes_aug, on=['idcomposicaopauta', 'idvotopauta'], how='left')
df_rec['datasessao'] = pd.to_datetime(df_rec['datasessao'], errors='coerce')
df_rec['n_recomendacoes'] = df_rec['recomendacoes'].apply(lambda x: len(x) if isinstance(x, list) else 0)


In [15]:
def remover_obrigacoes_duplicadas(obrigacoes_list):
    """
    Remove obrigações duplicadas de uma lista de NERObrigacao.

    Args:
        obrigacoes_list (list): A lista de objetos NERObrigacao.

    Returns:
        list: Uma nova lista com as obrigações únicas.
    """
    obrigacoes_vistas = set()
    obrigacoes_sem_duplicatas = []
    
    for obrigacao in obrigacoes_list:
        # Pega a descrição, remove espaços em branco e a converte para lowercase
        # para garantir que strings como 'Texto ' e 'texto' sejam tratadas como a mesma.
        descricao_limpa = obrigacao.descricao_obrigacao.strip().lower()
        
        if descricao_limpa not in obrigacoes_vistas:
            obrigacoes_vistas.add(descricao_limpa)
            obrigacoes_sem_duplicatas.append(obrigacao)
            
    return obrigacoes_sem_duplicatas

def remover_recomendacoes_duplicadas(recomendacoes_list):
    """
    Remove recomendações duplicadas de uma lista de NERRecomendacao.

    Args:
        recomendacoes_list (list): A lista de objetos NERRecomendacao.

    Returns:
        list: Uma nova lista com as recomendações únicas.
    """
    recomendacoes_vistas = set()
    recomendacoes_sem_duplicatas = []
    
    for recomendacao in recomendacoes_list:
        descricao_limpa = recomendacao.descricao_recomendacao.strip().lower()
        
        if descricao_limpa not in recomendacoes_vistas:
            recomendacoes_vistas.add(descricao_limpa)
            recomendacoes_sem_duplicatas.append(recomendacao)
            
    return recomendacoes_sem_duplicatas

In [ ]:
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session
# Assuming these are your imports

errors = []
k = 0

for _, row in df_ob.iterrows():
    print(f"Processando instância {row.name} de {len(df_ob)}")

    # Remove duplicatas da lista de obrigações antes de iniciar o loop
    obrigacoes_unicas = remover_obrigacoes_duplicadas(row['obrigacoes'])

    for obrigacao in obrigacoes_unicas:
        db_session = SessionLocal() # Create the session inside the loop to ensure a fresh connection for each row
        try:
            result_obrigacao = extract_obrigacao(row, obrigacao)
            insert_obrigacao(db_session, result_obrigacao, row)
            db_session.commit() # Commit the transaction here
        except IntegrityError:
            # This block handles the duplicate key error specifically
            print(f"Registro duplicado. Não será inserido: {row['idprocesso']}")
            db_session.rollback() # Rollback the transaction to clear the session state
        except Exception as e:
            # This block handles all other potential errors
            print(f"Erro ao processar obrigação: {e}")
            db_session.rollback() # Rollback on other errors as well
            errors.append((row['idprocesso'], obrigacao, str(e)))
        finally:
            db_session.close() # Always close the session to release the connection

In [37]:
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session
import traceback
# Assuming these are your imports

errors = []
k = 0

for _, row in df_rec.iterrows():
    print(f"Processando instância {row.name} de {len(df_rec)}")
    # Remove duplicatas da lista de recomendações antes de iniciar o loop
    recomendacoes_unicas = remover_recomendacoes_duplicadas(row['recomendacoes'])

    for recomendacao in recomendacoes_unicas:
        db_session = SessionLocal() # Create the session inside the loop to ensure a fresh connection for each row
        try:
            result_recomendacao = extract_recomendacao(row, recomendacao)
            insert_recomendacao(db_session, result_recomendacao, row)
            db_session.commit() # Commit the transaction here
        except IntegrityError:
            # This block handles the duplicate key error specifically
            print(f"Registro duplicado. Não será inserido: {row['idprocesso']}")
            db_session.rollback() # Rollback the transaction to clear the session state
        except Exception as e:
            # This block handles all other potential errors
            print(f"Erro ao processar recomendação: {e}")
            traceback.print_exc()
            db_session.rollback() # Rollback on other errors as well
            errors.append((row['idprocesso'], recomendacao, str(e)))
        finally:
            db_session.close() # Always close the session to release the connection

Processando instância 0 de 57
2025-11-18 15:26:21,410 INFO sqlalchemy.engine.Engine select @@version
2025-11-18 15:26:21,410 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-18 15:26:21,425 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-11-18 15:26:21,425 INFO sqlalchemy.engine.Engine [generated in 0.00070s] {}
2025-11-18 15:26:21,425 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-11-18 15:26:21,425 INFO sqlalchemy.engine.Engine [generated in 0.00081s] {}
2025-11-18 15:26:21,425 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-11-18 15:26:21,425 INFO sqlalchemy.engine.Engine [generated in 0.00045s] {}
2025-11-18 15:26:21,437 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-18 15:26:21,439 INFO sqlalchemy.engine.Engine INSERT INTO [Recomendacao] ([IdProcesso], [IdComposicaoPauta], [IdVotoPauta], [DescricaoRecomendacao], [PrazoCumprimentoRecomendaca

# From File

In [ ]:
all_decisions = pd.read_csv("dataset/all_decisions.csv")
all_decisions['year'] = pd.to_datetime(all_decisions['datasessao']).dt.year
all_decisions_2023 = all_decisions[all_decisions['year'] >= 2023]

In [ ]:
len(all_decisions_2023), len(all_decisions) 

In [ ]:
resultados = []
errors = []
for index, row in all_decisions_2023.iterrows():
    if index % 10 == 0 and index > 0:
        print(f"Processando instância {index} de {len(all_decisions_2023)}")
        all_decisions_2023.to_csv("dataset/all_decisions_2023_ner.csv", index=True)

    try:
        prompt_with_few_shot = generate_few_shot_ner_prompts(row['texto_acordao'])
        
        result = extractor_gpt4turbo.invoke(prompt_with_few_shot)
        all_decisions_2023.at[index, 'ner_decisao'] = result
        resultados.append(result)
    except Exception as e:
        errors.append({
            'index': index,
            'error': str(e),
            'row': row.to_dict()
        })

In [14]:
for e in errors:
    prompt_with_few_shot = generate_few_shot_ner_prompts(e['row']['texto_acordao'])
    result = extractor_gpt4turbo.invoke(prompt_with_few_shot)
    all_decisions_2023.at[e['index'], 'ner_decisao'] = result
    resultados.append(result)

In [16]:
all_decisions_2023.to_pickle("dataset/all_decisions_2023_ner.pkl")

# Creating Obrigacao database in local sqlite

In [306]:
Base = declarative_base()
engine = create_engine(
    f"mssql+pymssql://{os.getenv("SQL_SERVER_USER")}:{os.getenv("SQL_SERVER_PASS")}@{os.getenv("SQL_SERVER_HOST")}/{os.getenv("SQL_SERVER_DB")}",
    echo=True
)
#engine = create_engine("sqlite:///dataset/sqlite/obrigacoes.db", echo=True)
SessionLocal = sessionmaker(bind=engine)

In [307]:
class ObrigacaoORM(Base):
    __tablename__ = "Obrigacao"

    IdObrigacao = Column(Integer, primary_key=True, index=True)
    IdProcesso = Column(Integer, nullable=False)
    IdComposicaoPauta = Column(Integer, nullable=False)
    IdVotoPauta = Column(Integer, nullable=False)
    DescricaoObrigacao = Column(Text, nullable=False)
    DeFazer = Column(Boolean, default=True)
    Prazo = Column(String, nullable=True)
    DataCumprimento = Column(Date, nullable=True)
    OrgaoResponsavel = Column(String, nullable=True)
    IdOrgaoResponsavel = Column(Integer, nullable=True)
    TemMultaCominatoria = Column(Boolean, default=False)
    NomeResponsavelMultaCominatoria = Column(String, nullable=True)
    DocumentoResponsavelMultaCominatoria = Column(String, nullable=True)
    IdPessoaMultaCominatoria = Column(Integer, nullable=True)
    ValorMultaCominatoria = Column(Float, nullable=True)
    PeriodoMultaCominatoria = Column(String, nullable=True)
    EMultaCominatoriaSolidaria = Column(Boolean, default=False)
    SolidariosMultaCominatoria = Column(JSON, nullable=True)

In [ ]:
class RecomendacaoORM(Base):
    __tablename__ = "Recomendacao"

    IdRecomendacao = Column(Integer, primary_key=True, index=True)
    IdProcesso = Column(Integer, nullable=False)
    IdComposicaoPauta = Column(Integer, nullable=False)
    IdVotoPauta = Column(Integer, nullable=False)
    DescricaoRecomendacao = Column(Text, nullable=True)
    PrazoCumprimentoRecomendacao = Column(String, nullable=True)
    DataCumprimentoRecomendacao = Column(Date, nullable=True)
    NomeResponsavel = Column(String, nullable=True)
    IdPessoaResponsavel = Column(Integer, nullable=True)
    OrgaoResponsavel = Column(String, nullable=True)
    IdOrgaoResponsavel = Column(Integer, nullable=True)

    def __repr__(self):
        return f"<Recomendacao(IdRecomendacao={self.IdRecomendacao}, descricao_recomendacao='{self.descricao_recomendacao[:30]}...')>"

In [309]:
#Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(bind=engine)

2025-07-27 16:37:03,274 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-27 16:37:03,274 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(%(table_type_1)s AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(%(table_type_2)s AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(%(table_name_1)s AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(%(table_schema_1)s AS NVARCHAR(max))
2025-07-27 16:37:03,275 INFO sqlalchemy.engine.Engine [cached since 10.51s ago] {'table_type_1': 'BASE TABLE', 'table_type_2': 'VIEW', 'table_name_1': 'Obrigacao', 'table_schema_1': 'dbo'}
2025-07-27 16:37:03,313 INFO sqlalchemy.engine.Engine 
CREATE TABLE [Obrigacao] (
	[IdObrigacao] INTEGER NOT NULL IDENTITY, 
	[IdProcesso] INTEGER NOT NULL, 
	[IdComposicaoPauta] INTEGER NOT NULL, 
	[IdVotoPauta] INTEGER NOT NULL, 
	[De